In [111]:
# set parameters
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
pd.options.mode.chained_assignment = None  # default='warn'

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

In [102]:
# define functions
def find_nearest_loc(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def defensive_CR(df, HP, AC):
    n = df.shape[0]
    id = find_nearest_loc(df['HP Mean'], HP)
    CR_t = df.index[id]
    AC_t = df['AC'].loc[CR_t]
    id = int(max(0, min(n - 1, id + np.fix(0.5*(AC - AC_t)))))
    CR = df.index[id]
    return CR

def offensive_CR(df, DPR, AB):
    n = df.shape[0]
    id = find_nearest_loc(df['DPR Mean'], DPR)
    CR_t = df.index[id]
    AB_t = df['AB'].loc[CR_t]
    id = int(max(0, min(n - 1, id + np.fix(0.5*(AB - AB_t)))))
    CR = df.index[id]
    return CR

In [103]:
# import data
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','XP','AC','HP Mean','AB','DPR Mean'
dfD['eHP Mean']  = dfD['HP Mean' ]*np.power(1.05, dfD['AC'] - 12)
dfD['eDPR Mean'] = dfD['DPR Mean']*np.power(1.05, dfD['AB'] -  2)
dfD['eXP Mean'] = 0.25*dfD['eHP Mean']*dfD['eDPR Mean']
dfD.set_index('CR', inplace=True)


df0 = pd.read_csv('monster-cr-and-xp.csv')  # 'Book','Page','Type','Category','Legendary','CR','PB','XP',
                                            # 'HP','AC','adj HP','adj AC',
                                            # 'AB','adj AB','DPR', 'adj DPR'

df0['D-CR DMG'] = df0.apply(lambda row: defensive_CR(dfD, row['adj HP'], row['adj AC']), axis=1)
df0['O-CR DMG'] = df0.apply(lambda row: offensive_CR(dfD, row['adj DPR'], row['adj AB']), axis=1)
df0['C-CR DMG'] = 0.5*(df0['D-CR DMG'] + df0['O-CR DMG'])

df0['D-CR DMG Delta'] = df0['D-CR DMG'] - df0['CR']
df0['O-CR DMG Delta'] = df0['O-CR DMG'] - df0['CR']
df0['C-CR DMG Delta'] = df0['C-CR DMG'] - df0['CR']

df0['D-CR DMG unadj'] = df0.apply(lambda row: defensive_CR(dfD, row['HP'], row['AC']), axis=1)
df0['O-CR DMG unadj'] = df0.apply(lambda row: offensive_CR(dfD, row['DPR'], row['AB']), axis=1)
df0['C-CR DMG unadj'] = 0.5*(df0['D-CR DMG unadj'] + df0['O-CR DMG unadj'])
print('Includes {} monsters in database'.format(len(df0.index.to_list())))

Includes 1781 monsters in database


In [104]:
# Setup template and html config for plotly figures.
import plotly.graph_objects as go

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=False,
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(family='sans-serif', size=14),
        hovermode='closest',
        xaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        yaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            ticks='outside',
            minor=dict(
                ticks='outside',
            ),
            zeroline=False,
        ),
        hoverlabel=dict(align='left'),
    )
)

TFB_CONFIG = {
    'responsive': True, 
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}

def plot_confidence_interval(fig, df0, conf, xCol, yCol, name, line_color, fillcolor):
    # plot confidence
    if conf > 0:
        dfU = df0[[xCol, yCol]].groupby(xCol).quantile(0.5 + conf/2).reset_index()
        dfL = df0[[xCol, yCol]].groupby(xCol).quantile(0.5 - conf/2).reset_index()
        fig.add_trace(go.Scatter(x=dfU[xCol], y=dfU[yCol], line_width=0, legendgroup=name, showlegend=False, hoverinfo='skip'))
        fig.add_trace(go.Scatter(x=dfL[xCol], y=dfL[yCol], line_width=0, name=f'{name} - {conf:.0%} confidence', legendgroup=name, hoverinfo='skip', fill='tonexty', fillcolor=fillcolor))
    
    # plot mean
    dfM = df0[[xCol, yCol]].groupby(xCol).mean().reset_index()
    fig.add_trace(go.Scatter(
        x=dfM[xCol], y=dfM[yCol],
        mode='lines', 
        name=f'{name} - mean',
        legendgroup=name, 
        line_color=line_color,
        hovertemplate='<b>' + name + '</b><br>' + 'CR %{x:,.0f}<br>' + 'Mean %{y:,.1f}<br>' + '<extra></extra>'
    ))
    
def save_fig_html(fig, format, name):
    file_name = f'./{name}-{format}.html'

    match format:
        case 'large':
            fig.update_traces(marker=dict(size=6))
            fig.update_layout(font=dict(size=14))
        case 'small':
            fig.update_traces(marker=dict(size=4))
            fig.update_layout(font=dict(size=10))

    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = f'plotly-div-{format}'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [105]:
# plots monster HP vs CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='hit points',
            range=[0,1000],
            tick0=0, dtick=200,
            minor=dict(tick0=0, dtick=50),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

plot_confidence_interval(fig, df0, 0.0, 'CR', 'HP', 'HP', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

plot_confidence_interval(fig, df0, 0.0, 'CR', 'adj HP', 'adj HP', 'rgba(255, 0, 0, 1.0)', 'rgba(255, 0, 0, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=dfD.index, y=dfD['HP Mean'],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip', 
))

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    save_fig_html(fig, format='large', name='fig-hp-vs-cr')
    save_fig_html(fig, format='small', name='fig-hp-vs-cr')

In [99]:
# plots monster AC vs CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='armor class',
            range=[10,28],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

plot_confidence_interval(fig, df0, 0.0, 'CR', 'AC', 'AC', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

plot_confidence_interval(fig, df0, 0.0, 'CR', 'adj AC', 'adj AC', 'rgba(255, 0, 0, 1.0)', 'rgba(255, 0, 0, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=dfD.index, y=dfD['AC'],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip', 
))

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    save_fig_html(fig, format='large', name='fig-ac-vs-cr')
    save_fig_html(fig, format='small', name='fig-ac-vs-cr')

In [90]:
# defensive CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='defensive CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster confidence interval and average
plot_confidence_interval(fig, df0, 0.6, 'CR', 'D-CR DMG', 'D-CR', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=[0,30], y=[0,30],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip', 
    showlegend=False,
))

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    save_fig_html(fig, format='large', name='fig-d-cr-vs-cr')
    save_fig_html(fig, format='small', name='fig-d-cr-vs-cr')

In [112]:
# plots monster DPR vs CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='damage per round',
            range=[0,320],
            tick0=0, dtick=50,
            minor=dict(tick0=0, dtick=25),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

plot_confidence_interval(fig, df0, 0.0, 'CR', 'DPR', 'DPR', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

plot_confidence_interval(fig, df0, 0.0, 'CR', 'adj DPR', 'adj DPR', 'rgba(255, 0, 0, 1.0)', 'rgba(255, 0, 0, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=dfD.index, y=dfD['DPR Mean'],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip', 
))

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    save_fig_html(fig, format='large', name='fig-dpr-vs-cr')
    save_fig_html(fig, format='small', name='fig-dpr-vs-cr')

In [96]:
# plots monster AB vs CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='attack bonus',
            range=[0,20],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

plot_confidence_interval(fig, df0, 0.0, 'CR', 'AB', 'AB', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

plot_confidence_interval(fig, df0, 0.0, 'CR', 'adj AB', 'adj AB', 'rgba(255, 0, 0, 1.0)', 'rgba(255, 0, 0, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=dfD0.index, y=dfD0['AB'],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip', 
))

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    save_fig_html(fig, format='large', name='fig-ab-vs-cr')
    save_fig_html(fig, format='small', name='fig-ab-vs-cr')

In [91]:
# offensive CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='offensive CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster confidence interval and average
plot_confidence_interval(fig, df0, 0.6, 'CR', 'O-CR DMG', 'O-CR', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=[0,30], y=[0,30],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hoverinfo='skip', 
    showlegend=False,
))

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    save_fig_html(fig, format='large', name='fig-o-cr-vs-cr')
    save_fig_html(fig, format='small', name='fig-o-cr-vs-cr')

In [93]:
# final CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculated CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster confidence interval and average
plot_confidence_interval(fig, df0, 0.6, 'CR', 'C-CR DMG', 'DMG CR', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=[0,30], y=[0,30],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip', 
))

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    save_fig_html(fig, format='large', name='fig-dmg-cr-vs-cr')
    save_fig_html(fig, format='small', name='fig-dmg-cr-vs-cr')

In [110]:
# final CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculated CR',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster confidence interval and average
plot_confidence_interval(fig, df0, 0.6, 'CR', 'C-CR DMG unadj', 'unadjusted CR', 'rgba(0, 0, 255, 1.0)', 'rgba(0, 0, 255, 0.3)')

plot_confidence_interval(fig, df0, 0.6, 'CR', 'C-CR DMG', 'adjusted CR', 'rgba(255, 0, 0, 1.0)', 'rgba(255, 0, 0, 0.3)')

# plot reference line
fig.add_trace(go.Scatter(
    x=[0,30], y=[0,30],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip', 
))

# show figure
fig.show(config=TFB_CONFIG, width=600, height=450)
fig.update_layout(autosize=True)

# save figures
if SAVEFIGS:
    #save_fig_html(fig, format='large', name='fig-adj-cr-vs-cr')
    #save_fig_html(fig, format='small', name='fig-adj-cr-vs-cr')
    pass